In [19]:
import numpy as np
import scipy.io
from keras import layers

#from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
#from kt_utils import *
from sklearn.preprocessing import OneHotEncoder

import keras.backend as K
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

%matplotlib inline

In [4]:
x = scipy.io.loadmat("data_temp/S1-ADL1", mdict={'filled_features':'features', 'labels':'labels'})

In [29]:
X = x['filled_features']
Y = x['labels'][:,0]

print("X shape: ", X.shape)
print("Y shape: ", Y.shape)

X shape:  (45810, 113)
Y shape:  (45810,)


In [30]:
onehot_encoder = OneHotEncoder(sparse=False)
Y = onehot_encoder.fit_transform(x['labels'][:,0].reshape(-1, 1))
print("One hot encoder shape: ", Y.shape)

One hot encoder shape:  (45810, 5)


In [32]:
window_size = 50
stride = window_size / 2
num_features = X.shape[1]
samples, classes = Y.shape
print("Number of samples: ", samples, "\nNumber of classes: ", classes)

windows = int(samples // stride) - 1
print("Number of windows: ", windows)

Number of samples:  45810 
Number of classes:  5
Number of windows:  1831


In [33]:
cube = np.zeros([windows, window_size, 1, num_features])
cube.shape

(1831, 50, 1, 113)

In [37]:
lab_cum = np.zeros([windows, classes])
lab_cum.shape

(1831, 5)

In [45]:
for w in range(windows):
    index = int(w * stride)
    cube[w,:,:,:] = X[index:index+window_size, :].reshape((window_size,1,num_features))
    l = Y[index:index+window_size,:] # shape 50 x 5
    lab_cum[w,:] = np.sum(l, axis=0)/window_size

In [46]:
print('cumulative labels: ', lab_cum.shape, type(lab_cum), "\n", lab_cum)

cumulative labels:  (1831, 5) <class 'numpy.ndarray'> 
 [[0.   1.   0.   0.   0.  ]
 [0.   1.   0.   0.   0.  ]
 [0.   1.   0.   0.   0.  ]
 ...
 [1.   0.   0.   0.   0.  ]
 [1.   0.   0.   0.   0.  ]
 [0.52 0.   0.48 0.   0.  ]]


In [ ]:
def TestModel(input_shape):
    """ 
    Arguments:
    input_shape -- shape of the images of the dataset

    Returns: 
    model -- a Model() instance in Keras
    """
    
    
    model = Sequential()
    model.add(keras.layers.Conv1D(filters = 1,
                            kernel_size=50,
                            strides=25,
                            padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    model.add(Dropout(0.5))

    model.add(Convolution1D(nb_filter=8,
                            filter_length=4,
                            border_mode='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    model.add(Dropout(0.5))

    model.add(Flatten())

    model.add(Dense(64))
    model.add(BatchNormalization())
    model.add(LeakyReLU())

    model.add(Dense(2))
    model.add(Activation('softmax'))
    
    ### END CODE HERE ###
    
    return model